# Import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_NU M_THREADS"] = "2"
os.environ["OMP_NUM_THREADS"] = "2"
import time
from typing import Union
import numpy
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import Tensor
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from SSL.util.model_loader import load_model
from SSL.util.loaders import load_dataset, load_optimizer, load_callbacks, load_preprocesser
from SSL.util.checkpoint import CheckPoint, mSummaryWriter
from SSL.util.utils import reset_seed, get_datetime, track_maximum, DotDict, cache_to_disk, get_train_format, get_lr
from SSL.util.mixup import MixUpBatchShuffle
from SSL.loss import FixMatchLoss
from metric_utils.metrics import BinaryAccuracy, FScore, ContinueAverage, MAP
from augmentation_utils.spec_augmentations import SpecAugment
from torch.cuda import empty_cache
from torchsummary import summary

# Arguments

In [3]:
cfg = DotDict(
    hardware=DotDict(
        nb_cpu=10,
        nb_gpu=2,
    ),
    
    dataset=DotDict(
        dataset='audioset-unbalanced',
        num_classes=527,
    ),
    
    model=DotDict(
        model="MobileNetV2",
    ),
    
    train_param=DotDict(
        supervised_ratio=0.1,
        batch_size=128,
        nb_iteration=250000,
        learning_rate=0.001,
        seed=1234,
        resume=False,
        
        train_folds=None,
        val_folds=None,
    ),
    
    fixmatch=DotDict(
        mask_threshold=0.1,
        guess_threshold=0.75,
        lambda_s=1.0,
        lambda_u=1.0,
    ),
    
    mixup=DotDict(
        use=False,
        alpha=1.0,
        max=True,
        label=True
    ),
    
    specaugment=DotDict(
        use=False,
        time_drop_width=32,
        time_stripe_num=1,
        freq_drop_width=4,
        freq_stripe_num=1,
    ),
)

cfg['path'] = DotDict(
    dataset_root='../../datasets',
    checkpoint_root='../../model_save',
    tensorboard_root='../../tensorboard',
)
cfg.path['checkpoint_path']=f'{cfg.path.checkpoint_root}/{cfg.dataset.dataset}/fixmatch'
cfg.path['tensorboard_path']=f'{cfg.path.tensorboard_root}/{cfg.dataset.dataset}/fixmatch'

# reset the seed

In [4]:
reset_seed(cfg.train_param.seed)

 # -------- Get the pre-processer --------

In [5]:
train_transform, val_transform = load_preprocesser(cfg.dataset.dataset, "fixmatch")

/users/samova/lcances/.miniconda3/envs/pytorch-dev/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


# -------- Get the dataset --------

In [6]:
_, train_loader, val_loader = load_dataset(
    cfg.dataset.dataset,
    "fixmatch",

    dataset_root=cfg.path.dataset_root,
    supervised_ratio=cfg.train_param.supervised_ratio,
    batch_size=cfg.train_param.batch_size,
    train_folds=cfg.train_param.train_folds,
    val_folds=cfg.train_param.val_folds,

    train_transform=train_transform,
    val_transform=val_transform,

    num_workers=cfg.hardware.nb_cpu,  # With the cache enable, it is faster to have only one worker
    pin_memory=True,
    seed=cfg.train_param.seed,

    verbose=1
)

/users/samova/lcances/semi-supervised/SSL/dataset/speechcommands.py
creating validation dataset
creating weak and strong augmented train dataset
spliting the dataset
cache path:  .cache_supervised_ratio=0.1_unsupervised_ratio=0.9_batch_size=128_verbose=True
split ready, loading cache file
Getting all target
Sort the classes
merging dataloaders


In [7]:
# The input shape of the data is used to generate the model
input_shape = tuple(train_loader._iterables[0].dataset[0][0].shape)

/users/samova/lcances/.miniconda3/envs/pytorch-dev/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore
/users/samova/lcances/.miniconda3/envs/pytorch-dev/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore


# -------- Prepare the model --------

In [8]:
torch.cuda.empty_cache()
model_func = load_model(cfg.dataset.dataset, cfg.model.model)
model = model_func(input_shape=input_shape, num_classes=cfg.dataset.num_classes)
model = model.cuda()
summary(model, input_shape)

if cfg.hardware.nb_gpu > 1:
    model = nn.DataParallel(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 32, 64, 501]             288
         AvgPool2d-2          [-1, 32, 32, 250]               0
       BatchNorm2d-3          [-1, 32, 32, 250]              64
             ReLU6-4          [-1, 32, 32, 250]               0
            Conv2d-5          [-1, 32, 32, 250]             288
         AvgPool2d-6          [-1, 32, 32, 250]               0
       BatchNorm2d-7          [-1, 32, 32, 250]              64
             ReLU6-8          [-1, 32, 32, 250]               0
            Conv2d-9          [-1, 16, 32, 250]             512
      BatchNorm2d-10          [-1, 16, 32, 250]              32
 InvertedResidual-11          [-1, 16, 32, 250]               0
           Conv2d-12          [-1, 96, 32, 250]           1,536
      BatchNorm2d-13          [-1, 96, 32, 250]             192
            ReLU6-14          [-1, 96, 

# -------- Tensorboard and checkpoint --------

In [9]:
# Prepare suffix
# normale training parameters
sufix_title = ''
sufix_title += f'_{cfg.train_param.learning_rate}-lr'
sufix_title += f'_{cfg.train_param.supervised_ratio}-sr'
sufix_title += f'_{cfg.train_param.nb_epoch}-e'
sufix_title += f'_{cfg.train_param.batch_size}-bs'
sufix_title += f'_{cfg.train_param.seed}-seed'

# mixup parameters
if cfg.mixup.use:
    sufix_title += '_mixup'
    if cfg.mixup.max: sufix_title += "-max"
    if cfg.mixup.label: sufix_title += "-label"
    sufix_title += f"-{cfg.mixup.alpha}-a"

# SpecAugment parameters
if cfg.specaugment.use:
    sufix_title += '_specAugment'
    sufix_title += f'-{cfg.specaugment.time_drop_width}-tdw'
    sufix_title += f'-{cfg.specaugment.time_stripe_num}-tsn'
    sufix_title += f'-{cfg.specaugment.freq_drop_width}-fdw'
    sufix_title += f'-{cfg.specaugment.freq_stripe_num}-fsn'

# -------- Tensorboard logging --------

In [10]:
tensorboard_title = f'{get_datetime()}_{cfg.model.model}_{sufix_title}'
log_dir = f'{cfg.path.tensorboard_path}/{tensorboard_title}'
print('Tensorboard log at: ', log_dir)

tensorboard = mSummaryWriter(log_dir=log_dir, comment=model_func.__name__)

Tensorboard log at:  ../../tensorboard/audioset-unbalanced/fixmatch/2021-03-17_15:47:04_MobileNetV2__0.001-lr_0.1-sr_None-e_128-bs_1234-seed


# -------- Optimizer, callbacks, loss and checkpoint --------

In [11]:
optimizer = load_optimizer(cfg.dataset.dataset, 'fixmatch', learning_rate=cfg.train_param.learning_rate, model=model)
callbacks = load_callbacks(cfg.dataset.dataset, 'fixmatch', optimizer=optimizer, nb_epoch=cfg.train_param.nb_iteration)
loss_fixmatch = FixMatchLoss()
loss_bce = nn.BCEWithLogitsLoss(reduction="mean")

checkpoint_title = f'{cfg.model.model}_{sufix_title}'
checkpoint_path = f'{cfg.path.checkpoint_path}/{checkpoint_title}'
checkpoint = CheckPoint(model, optimizer, mode="max", name=checkpoint_path)

checkpoint initialise at:  /users/samova/lcances/semi-supervised/model_save/audioset-unbalanced/fixmatch/MobileNetV2__0.001-lr_0.1-sr_None-e_128-bs_1234-seed
name:  MobileNetV2__0.001-lr_0.1-sr_None-e_128-bs_1234-seed
mode:  max


# -------- Metrics and print formater --------

In [12]:
metrics = DotDict(
    fscore_s=FScore(),
    fscore_u=FScore(),
    acc_s=BinaryAccuracy(),
    acc_u=BinaryAccuracy(),
    avg_fn=ContinueAverage(),
)

val_metrics = DotDict(
    fscore_s=FScore(),
    acc_s=BinaryAccuracy(),
    mAP_fn=MAP(),
    avg_fn=ContinueAverage(),
)

maximum_tracker = track_maximum()

In [13]:
header, train_formater, val_formater = get_train_format('audioset-fixmatch')

# -------- Augmentations ---------

In [14]:
spec_augmenter = SpecAugment(time_drop_width=cfg.specaugment.time_drop_width,
                                  time_stripes_num=cfg.specaugment.sa_time_stripe_num,
                                  freq_drop_width=cfg.specaugment.freq_drop_width,
                                  freq_stripes_num=cfg.specaugment.req_stripe_num)

mixup_fn = MixUpBatchShuffle(alpha=cfg.mixup.alpha, apply_max=cfg.mixup.max, mix_labels=cfg.mixup.label)

# -------- Training and Validation function --------

In [15]:
activation = nn.Sigmoid()

In [16]:
def guess_label(x_uw: Tensor) -> (Tensor, Tensor):    
    logits_uw = model(x_uw)
    pred_uw = activation(logits_uw)

    nb_classes = cfg.dataset.num_classes
    labels_u = (pred_uw > cfg.fixmatch.guess_threshold).long()
#     labels_u = F.one_hot(pred_uw, cfg.dataset.num_classes)
    return labels_u, pred_uw


def confidence_mask(pred: Tensor, threshold: float, dim: int) -> Tensor:
    max_values, _ = pred.max(dim=dim)
    return (max_values > threshold).float()


m_ = lambda x: x.mean(size=100)

In [17]:
def train(epoch, Sw, Uw, Us, start_time):
    # aliases
    M = metrics
    T = tensorboard.add_scalar
    
    model.train()
    
    x_sw, y_sw = Sw  # Supervised weak augmented
    x_uw, y_uw = Uw  # Unsupervised weak augmented
    x_us, y_us = Us  # Unsupervised strong augmented
    
    x_sw, y_sw = x_sw.cuda(), y_sw.cuda()
    x_uw, y_uw = x_uw.cuda(), y_uw.cuda()
    x_us, y_us = x_us.cuda(), y_us.cuda()
    
    # Apply mixup is needed
    if cfg.mixup.use:
        pass
    
    # Apply specaugment if needed
    if cfg.specaugment.use:
        pass
    
    # Use guess u label with prediction of weak augmentation of u
    with torch.no_grad():
        pseudo_y_uw, pred_uw = guess_label(x_uw)
        mask = confidence_mask(pseudo_y_uw, cfg.fixmatch.mask_threshold, dim=1)

    optimizer.zero_grad()
    
    # Compute predictions
    logits_sw = model(x_sw)
    logits_us = model(x_us)

    pred_sw = activation(logits_sw)
    pred_us = activation(logits_us)

    # Update model
    loss, loss_s, loss_u = loss_fixmatch(
        pred_sw,
        pred_us,
        mask,
        y_sw,
        pseudo_y_uw,
        lambda_s=cfg.fixmatch.lambda_s,
        lambda_u=cfg.fixmatch.lambda_u,
    )

    loss.backward()
    optimizer.step()

    # Compute metrics
    with torch.no_grad():
        fscore_s = M.fscore_s(pred_sw, y_sw)
        acc_s = M.acc_s(pred_sw, y_sw)
        fscore_u = M.fscore_u(pred_uw, y_uw)  # Use true label for monitoring purpose
        acc_u = M.fscore_u(pred_uw, y_uw)  # Use true label for monitoring purpose
        avg_ce = M.avg_fn(loss.item())
        
        # logs
        print(train_formater.format(
            "Training: ",
            epoch + 1,
            e, cfg.train_param.nb_iteration,
            "", m_(avg_ce),
            "", m_(acc_s), m_(fscore_s), m_(acc_u), m_(fscore_u), 0.0,
            time.time() - start_time,
        ), end="\r")
        
        T("train/loss",   loss.item(), epoch)
        T("train/loss_s", loss_s.item(), epoch)
        T("train/loss_u", loss_u.item(), epoch)
        T("train/labels_used", mask.mean().item(), epoch)
        
        T('train/fscore_s', m_(fscore_s), epoch)
        T('train/fscore_u', m_(fscore_u), epoch)
        T('train/acc_s', m_(acc_s), epoch)
        T('train/acc_u', m_(acc_u), epoch)

In [18]:
def val(epoch):
    # aliases
    M = val_metrics
    T = tensorboard.add_scalar
    nb_batch = len(val_loader)

    start_time = time.time()
    print("")
    
    model.eval()
    
    with torch.set_grad_enabled(False):
        for i, (X, y) in enumerate(val_loader):
            X = X.cuda().float()
            y = y.cuda().float()

            logits = model(X)
            loss = loss_bce(logits, y)

            pred = torch.sigmoid(logits)
            fscore = M.fscore_s(pred, y)
            acc = M.acc_s(pred, y)
            mAP = M.mAP_fn(pred.cpu().reshape(-1), y.cpu().reshape(-1))
            avg_ce = M.avg_fn(loss.item())

            # logs
            print(val_formater.format(
                "Validation: ",
                epoch + 1,
                i, nb_batch,
                "", m_(avg_ce),
                "", m_(acc), m_(fscore), 0.0, 0.0, m_(mAP),
                time.time() - start_time
            ), end="\r")

    T("val/Lce", m_(avg_ce), epoch)
    T("val/f1", m_(fscore), epoch)
    T("val/acc", m_(acc), epoch)
    T("val/mAP", m_(mAP), epoch)

    T("hyperparameters/learning_rate", get_lr(optimizer), epoch)

    T("max/acc", maximum_tracker("acc", m_(acc)), epoch)
    T("max/f1", maximum_tracker("f1", m_(fscore)), epoch)
    T('max/mAP', maximum_tracker('mAP', m_(mAP)), epoch)

    return avg_ce, fscore, mAP

In [ ]:
# -------- Training loop --------
if cfg.train_param.resume:
    checkpoint.load_last()

start_iteration = checkpoint.epoch_counter
end_iteration = cfg.train_param.nb_iteration

train_iterator = iter(train_loader)
start_time = time.time()

print(header)

for e in range(start_iteration, end_iteration):
    # Validation every 500 iteration
    if e % 500 == 0:
        val_avg_ce, val_fscore, val_mAP = val(e)
        print('')
        checkpoint.step(m_(val_mAP))
        tensorboard.flush()

    # Perform train
    train(e, *next(train_iterator), start_time)

.                Epoch -       /       - Losses: ce        - metrics: acc_s        F1_s         acc_u        F1_u         mAP          - Time  

Validation:      1     -   159 / 160   -         0.6935    -          5.094e-01    1.029e-02    0.000e+00    0.000e+00    5.102e-03    - 60.0909
Training:        500   -   499 / 250000 -         0.0000    -          5.851e-03    7.891e-03    6.526e-03    6.526e-03    0.000e+00    - 695.0684
Validation:      501   -   159 / 160   -         63.5126   -          6.145e-03    8.485e-03    0.000e+00    0.000e+00    4.260e-03    - 63.7243
Training:        1000  -   999 / 250000 -         0.0000    -          3.921e-03    7.811e-03    6.388e-03    6.388e-03    0.000e+00    - 1340.6074
Validation:      1001  -   159 / 160   -         65.5443   -          4.257e-03    8.478e-03    0.000e+00    0.000e+00    4.257e-03    - 65.6587
Training:        1500  -  1499 / 250000 -         0.0000    -          3.951e-03    7.871e-03    6.393e-03    6.393e-03    0.

# -------- Save the hyper parameters and the metrics --------

In [ ]:
hparams = {
    'dataset': cfg.dataset.dataset,
    'model': cfg.model.model,
    'supervised_ratio': cfg.train_param.supervised_ratio,
    'batch_size': cfg.train_param.batch_size,
    'nb_iteration': cfg.train_param.nb_iteration,
    'learning_rate': cfg.train_param.learning_rate,
    'seed': cfg.train_param.seed,
    
    'threshold': cfg.fixmatch.threshold,
    'lambda_s': cfg.fixmatch.lambda_s,
    'lambda_u': cfg.fixmatch.lambda_u,
    
    'mixup': cfg.mixup.use,
    'mixup-alpha': cfg.mixup.alpha,
    'mixup-max': cfg.mixup.max,
    'mixup-label': cfg.mixup.label,
    
    'specaugment(sa)': cfg.specaugment.use,
    'sa_time_drop_width': cfg.specaugment.time_drop_width,
    'sa_time_stripe_num': cfg.specaugment.time_stripe_num,
    'sa_freq_drop_width': cfg.specaugment.freq_drop_width,
    'sa_freq_stripe_num': cfg.specaugment.freq_stripe_num,
}

# convert all value to str
hparams = dict(zip(hparams.keys(), map(str, hparams.values())))

final_metrics = {
    "max_acc": maximum_tracker.max["acc"],
    "max_f1": maximum_tracker.max["f1"],
}

tensorboard.add_hparams(hparams, final_metrics)

tensorboard.flush()
tensorboard.close()